### HOW TO CACHE YOUR STUFF

Experimental!  Quantifying [Issue 12](https://github.com/BrownDwarf/gollum/issues/12)

In [1]:
from gollum.phoenix import PHOENIXGrid, PHOENIXSpectrum
import numpy as np
import astropy.units as u
from time import time

In [35]:
%%time
grid = PHOENIXGrid(teff_range=None, logg_range=None, metallicity_range=(0, 0))

Processing Teff=12000K|log(g)=6.00|Z=+0.0: 100%|█| 657/657 [00:34<00:00, 18.81it


CPU times: user 6.22 s, sys: 6.99 s, total: 13.2 s
Wall time: 49.7 s


In [2]:
#%%time
#grid = PHOENIXGrid(teff_range=(2500,10000), logg_range=(1, 5), metallicity_range=(0, 0.5))

Processing Teff=10000K|log(g)=5.00|Z=+0.5: 100%|█| 854/854 [00:33<00:00, 25.25it


CPU times: user 7.96 s, sys: 6.18 s, total: 14.1 s
Wall time: 41 s


In [36]:
%%time
flux_array = np.vstack([spec.flux.value for spec in grid])
np.save("flux_array.npy", flux_array)

CPU times: user 788 ms, sys: 3.62 s, total: 4.41 s
Wall time: 7.91 s


In [4]:
%%time
flux_array2 = np.load("flux_array.npy")

CPU times: user 862 µs, sys: 551 ms, total: 552 ms
Wall time: 2.18 s


In [8]:
print(.758/55.6)

0.013633093525179855


In [9]:
fixed_axis = grid[0].spectral_axis
fixed_unit = grid[0].flux.unit

In [10]:
grid[0].flux.unit

Unit("erg / (cm3 s)")

In [11]:
%%time
spec_list = [PHOENIXSpectrum(flux=flux_array2[i,:]*fixed_unit, spectral_axis=fixed_axis) for i in range(grid.n_spectra)]

CPU times: user 1.32 s, sys: 2.14 s, total: 3.47 s
Wall time: 9.75 s


In [12]:
from specutils import SpectrumList, SpectrumCollection

In [13]:
flux_array2.T.shape, fixed_axis.shape

((338649, 854), (338649,))

In [ ]:
spectral_axis_2D = np.repeat([fixed_axis.value], len(grid), axis=0)*u.Angstrom

In [32]:
SpectrumCollection(flux = flux_array2*fixed_unit, spectral_axis=spectral_axis_2D)

SpectrumCollection(ndim=1, shape=(854,))
    Flux units:          erg / (cm3 s)
    Spectral axis units: Angstrom
    Uncertainty type:    None

## Finale: "Round trip" to the grid

Technically, we are still missing metadata though, so this implementation experiment is **incomplete**!

In [34]:
%%time
grid2=PHOENIXGrid(flux=flux_array2*fixed_unit, 
                    spectral_axis = np.repeat([fixed_axis.value], 854, axis=0)*u.Angstrom, )

it worked!
CPU times: user 403 ms, sys: 2.54 s, total: 2.94 s
Wall time: 7.46 s


In [37]:
flux_array2[1,:].shape

(338649,)